Para esta función se necesita:

- Desde el Dataframe user_items el id de juego y la cantidad de tiempo jugado
- Desde el Dataframe steam_games el id de juego, el género y el año de lanzamiento

Con estos datos se crea el Dataframe que se sube para la consulta

In [2]:
# Importar las librerías a utilizar
import pandas as pd

In [116]:
# Cargar los Dataframes
df_user_items = pd.read_csv("C:/Users/Usuario/Desktop/Varios yo/SoyHenry/P1/Henry_PI1_Steam/data/user_items_2.csv")
df_steam_games = pd.read_csv("C:/Users/Usuario/Desktop/Varios yo/SoyHenry/P1/Henry_PI1_Steam/data/steam_games_2.csv")

## TRANSFORMACIONES de user_items

In [117]:
# Reviso las columnas de user_items
df_user_items.columns

Index(['item_id', 'item_name', 'playtime_forever', 'user_id'], dtype='object')

In [118]:
# Desde el Dataframe df_user_items elimino las columnas que no necesito
df_user_items.drop("item_name", axis=1, inplace=True)
df_user_items.drop("user_id", axis=1, inplace=True)
df_user_items

,item_id,playtime_forever
0,10,6
1,20,0
2,30,7
3,40,0
4,50,0
...,...,...
5153204,346330,0
5153205,373330,0
5153206,388490,3
5153207,521570,4


In [119]:
# Reviso el Dataframe
df_user_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 2 columns):
 #   Column            Dtype
---  ------            -----
 0   item_id           int64
 1   playtime_forever  int64
dtypes: int64(2)
memory usage: 78.6 MB


In [120]:
# Para evitar inconvenientes, convierto el id de user_items al mismo formato del id de steam_games
df_user_items["item_id"] = df_user_items["item_id"].astype(int).astype(float)

In [121]:
# Por optimización, eliminar los registros con "0" horas jugadas, ya que necesito sumar horas jugadas y estos no suman
df_user_items = df_user_items[df_user_items['playtime_forever'] != 0]

## TRANSFORMACIONES de steam_games

In [122]:
# Reviso las columnas de steam_games
df_steam_games.columns

Index(['Unnamed: 0', 'app_name', 'release_date', 'price', 'early_access', 'id',
       'developer', 'genre', 'tag', 'spec'],
      dtype='object')

In [123]:
# Desde el Dataframe df_steam_games elimino las columnas que no necesito
df_steam_games.drop("Unnamed: 0", axis=1, inplace=True)
df_steam_games.drop("app_name", axis=1, inplace=True)
df_steam_games.drop("price", axis=1, inplace=True)
df_steam_games.drop("early_access", axis=1, inplace=True)
df_steam_games.drop("developer", axis=1, inplace=True)
df_steam_games.drop("tag", axis=1, inplace=True)
df_steam_games.drop("spec", axis=1, inplace=True)
df_steam_games

,release_date,id,genre
0,2018-01-04,761140.0,Action
1,2018-01-04,761140.0,Action
2,2018-01-04,761140.0,Action
3,2018-01-04,761140.0,Action
4,2018-01-04,761140.0,Action
...,...,...,...
2088821,NaN,681550.0,NaN
2088822,NaN,681550.0,NaN
2088823,NaN,681550.0,NaN
2088824,NaN,681550.0,NaN


In [124]:
# Debido a la presenca de "tag" y "spec" ahora eliminadas, quedaron muchos duplicados
df_steam_games.drop_duplicates(inplace=True)

In [125]:
# Reviso el Dataframe
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74836 entries, 0 to 2088778
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   release_date  71937 non-null  object 
 1   id            74833 non-null  float64
 2   genre         71553 non-null  object 
dtypes: float64(1), object(2)
memory usage: 2.3+ MB


In [126]:
# Extraer solo el año de lanzamiento de la fecha de lanzamiento

df_steam_games["release_date"] = df_steam_games["release_date"].str.split('-').str[0]

In [127]:
df_steam_games["release_date"].unique()

array(['2018', '2017', nan, '1997', '1998', '2016', '2006', '2005',
       '2003', '2007', '2002', '2000', '1995', '1996', '1994', '2001',
       '1993', '2004', '1999', '2008', '2009', '1992', '1989', '2010',
       '2011', '2013', '2012', '2014', '1983', '1984', '2015', '1990',
       '1988', '1991', '1985', '1982', '1987', '1981', '1986', '2021',
       '2019', '1975', '1970', '1980'], dtype=object)

-------------------------------

### Ahora con los Dataframes procesados, creo el csv para la consulta

In [129]:
# Sumar los minutos jugados para cada juego
df_minutos_por_juego = df_user_items.groupby(['item_id'])['playtime_forever'].sum()

In [130]:
df_minutos_por_juego

item_id
10.0        17386015
20.0          961702
30.0          758991
40.0          154486
50.0          734562
              ...   
527570.0           2
527810.0           2
527890.0           1
527900.0          44
528660.0         182
Name: playtime_forever, Length: 10050, dtype: int64

In [131]:
# Utilizar la función merge para unir los DataFrames por las columnas "item_id" y "id"
df_steam_games_mas_tiempo = df_steam_games.merge(df_minutos_por_juego, left_on='id', right_on='item_id', how='inner')

In [132]:
df_steam_games_mas_tiempo

,release_date,id,genre,playtime_forever
0,1997,282010.0,Action,9319
1,1997,282010.0,Indie,9319
2,1997,282010.0,Racing,9319
3,1998,70.0,Action,2682852
4,2006,1640.0,Strategy,27397
...,...,...,...,...
20522,2003,200980.0,Strategy,3393
20523,2003,200980.0,RPG,3393
20524,2003,200980.0,Indie,3393
20525,2004,13230.0,Action,349119


In [133]:
# Agrupar los datos por género y año de lanzamiento, y sumar las horas jugadas
df_genero_lanzamiento_minutos = df_steam_games_mas_tiempo.groupby(['genre', 'release_date'])['playtime_forever'].sum().reset_index()

In [134]:
df_genero_lanzamiento_minutos

,genre,release_date,playtime_forever
0,Action,1983,3582
1,Action,1984,384
2,Action,1988,16243
3,Action,1989,607
4,Action,1990,18787
...,...,...,...
335,Web Publishing,2013,335849
336,Web Publishing,2014,33732
337,Web Publishing,2015,348861
338,Web Publishing,2016,136


In [135]:
# Para cada género, mostrar el año que corresponde con el mayor tiempo jugado
df_PlayTimeGenre = df_genero_lanzamiento_minutos.loc[df_genero_lanzamiento_minutos.groupby('genre')['playtime_forever'].idxmax()]

df_PlayTimeGenre

,genre,release_date,playtime_forever
26,Action,2012,1099062852
59,Adventure,2011,224347981
71,Animation &amp; Modeling,2015,1354685
75,Audio Production,2014,466246
104,Casual,2015,82515351
109,Design &amp; Illustration,2012,1947406
116,Early Access,2013,120366296
124,Education,2013,342804
141,Free to Play,2013,147728688
158,Indie,2006,452672790


In [115]:
# Exportar el Dataframe a .csv eliminando el índice
df_PlayTimeGenre.to_csv("PlayTimeGenre.csv", index=False)

In [3]:
# Verificar el .csv
df = pd.read_csv("PlayTimeGenre.csv")
df

,genre,release_date,playtime_forever
0,Action,2012,1099062852
1,Adventure,2011,224347981
2,Animation &amp; Modeling,2015,1354685
3,Audio Production,2014,466246
4,Casual,2015,82515351
5,Design &amp; Illustration,2012,1947406
6,Early Access,2013,120366296
7,Education,2013,342804
8,Free to Play,2013,147728688
9,Indie,2006,452672790
